__1. Gazetteer Data__

__*a. Create Unmanaged Tables*__

The first step of this assignment involves loading the data from the CSV files, combining the file with the file for the other year, and saving it to disk as a table. The following code should provide a template to help you combine tables and save them to the warehouse directory. Click on the image to download the sample code.

For each CSV file in the 2017 and 2018 directories, load the data into Spark, combine it with the corresponding data from the other year and save it to disk. Once you have finished saving all of the files as tables, verify that you have loaded the files properly by loading the tables into Spark, and performing a simple row count on each table.

The following Python code should provide you a template for loading the tables as an external table in Spark. Click on the image to download the sample code.

As stated previously, in a typical Hadoop distribution, you could save these tables as persistent tables in Apache Hive, but since we are not introducing Hive in this class, we need to load these tables into Spark and query them using SQL within Python.

The following code shows how to count the number of rows in the places table and show the results. Click on the image to download the code.

As an aside, spark.catalog module offers useful utility functions such as spark.catalog.listTables() to list all of the currently available tables. These are useful for inspecting the Spark SQL warehouse.

__*b. Load and Query Tables*__

Now that we have saved the data to external tables, we will load the tables back into Spark and create a report using Spark SQL. For this report, we will create a report on school districts for the states of Nebraska and Iowa using the elementary_schools, secondary_schools and unified_school_districts tables. Using Spark SQL, create a report with the following information.

This table contains the number of elementary, secondary, and unified school districts in each state for each year. Note that the numbers in this table are notional and do not represent the actual results.

In [2]:
# Create necessary directories
warehouse_dir = "/FileStore/spark-warehouse"
dbutils.fs.mkdirs(warehouse_dir) 
dbutils.fs.mkdirs("/FileStore/2017") 
dbutils.fs.mkdirs("/FileStore/2018") 

display(dbutils.fs.ls("/FileStore"))

path,name,size
dbfs:/FileStore/2017/,2017/,0
dbfs:/FileStore/2018/,2018/,0
dbfs:/FileStore/df/,df/,0
dbfs:/FileStore/import-stage/,import-stage/,0
dbfs:/FileStore/sortDF.csv/,sortDF.csv/,0
dbfs:/FileStore/spark-warehouse/,spark-warehouse/,0
dbfs:/FileStore/tables/,tables/,0


After this, I manually uploaded the `places.csv` files from 2017 and 2018, following the steps below:

1. Upload the 2017 file to dbfs

2. Move the 2017 file to 2017 directory

  _dbutils.fs.cp("/FileStore/tables/places.csv", "/FileStore/2017/places.csv")_

3. Remove the 2017 file from dbfs

  _dbutils.fs.rm("/FileStore/tables/places.csv")_
  
4. Move the 2018 file to 2018 directory

  _dbutils.fs.cp("/FileStore/tables/places.csv", "/FileStore/2018/places.csv")_

5. Remove the 2018 file from dbfs

  _dbutils.fs.rm("/FileStore/tables/places.csv")_
  
I had to do it this way since it was the only known method to me at the time of this excercise.

In [4]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession\
  .builder\
  .appName('DSC650Assignment5')\
  .config("spark.sql.warehouse.dir", warehouse_dir)\
  .getOrCreate()

In [5]:
# Define data file path
csv_file_path2017 = '/FileStore/2017/places.csv'
csv_file_path2018 = '/FileStore/2018/places.csv'

# Load data files
df2017 = spark.read.load(
  csv_file_path2017,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

df2018 = spark.read.load(
  csv_file_path2018,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Check data volume for both files
print('2017 file row count:', df2017.count())
print('2018 file row count:', df2018.count())

2017 file row count: 29577
2018 file row count: 29574

In [6]:
# Union the 2 data files
places = df2017.unionAll(df2018)

# Check total row count
print('Total row count:', places.count())

# Save data to table in default database
places.write.saveAsTable('places')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3983604114395494> in <module> () 
 6 
 7 # Save data to table in default database 
 ----> 8 places . write . saveAsTable ( 'places' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable (self, name, format, mode, partitionBy, **options) 
 775 if format is not None : 
 776 self . format ( format ) 
 --> 777 self . _jwrite . saveAsTable ( name ) 
 778 
 779 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'Table `places` already exists.;'

In [7]:
# I am going so save the dataframes as external table in the warehouse
df2017.write.option("path", warehouse_dir).saveAsTable("df2017", mode='overwrite')

df2018.write.option("path", warehouse_dir).saveAsTable("df2018", mode='overwrite')

places.write.option("path", warehouse_dir).saveAsTable("places", mode='overwrite')

In [8]:
# Query the data using the sqlContext method
sqlContext.sql("SELECT COUNT(*) FROM places").show()

+--------+
count(1)|
+--------+
 59151|
+--------+

In [9]:
# Query the data using the spark.sql method - a preferred one for Spark 2x
spark.sql("SELECT COUNT(*) FROM places").show()

+--------+
count(1)|
+--------+
 21779|
+--------+

In [10]:
# load school district data for 2017 and 2018
# Define data file path
path2017 = '/FileStore/2017/'
path2018 = '/FileStore/2018/'

elementary_file_2017 = path2017+'elementary_schools.csv'
secondary_file_2017 = path2017+'secondary_schools.csv'
unified_file_2017 = path2017+'unified_school_districts.csv'

elementary_file_2018 = path2018+'elementary_schools.csv'
secondary_file_2018 = path2018+'secondary_schools.csv'
unified_file_2018 = path2018+'unified_school_districts.csv'

# Load data files - 2017
# Elementary
elementary2017 = spark.read.load(
  elementary_file_2017,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Secondary
secondary2017 = spark.read.load(
  secondary_file_2017,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Unified school district
unified2017 = spark.read.load(
  unified_file_2017,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Load data files - 2018
# Elementary
elementary2018 = spark.read.load(
  elementary_file_2018,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Secondary
secondary2018 = spark.read.load(
  secondary_file_2018,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

# Unified school district
unified2018 = spark.read.load(
  unified_file_2018,
  format='csv',
  sep=',',
  inferSchema=True,
  header=True
)

In [11]:
# Union the 2 data files
elementary = elementary2017.unionAll(elementary2018)
secondary = secondary2017.unionAll(secondary2018)
unified = unified2017.unionAll(unified2018)

# Check total row count
print('Total elementary row count:', elementary.count())
print('Total secondary row count:', secondary.count())
print('Total unified row count:', unified.count())

# Save data to table in default database
elementary.write.saveAsTable('elementary')
secondary.write.saveAsTable('secondary')
unified.write.saveAsTable('unified')

Total elementary row count: 3926
Total secondary row count: 974
Total unified row count: 21779

In [12]:
# I am going so save the dataframes as external table in the warehouse
elementary.write.option("path", warehouse_dir).saveAsTable("elementary", mode='overwrite')
secondary.write.option("path", warehouse_dir).saveAsTable("secondary", mode='overwrite')
unified.write.option("path", warehouse_dir).saveAsTable("unified", mode='overwrite')

I encountered an errror while adding the secondary data. The data contain mixed data type in `water_area_meters_sq` field (integer and int64). I have researched and found some sort of explanation in this link: (https://stackoverflow.com/questions/50383360/parquet-datatype-issue).

For the time being, I have proceeded with only elementary and unified data.

In [14]:
display(spark.sql("SELECT a.state as State, a.year as Year, a.cnt Elementary, b.cnt Unified \
                   FROM (SELECT state, year, count(*) as cnt \
                         FROM elementary \
                         WHERE state IN('NE','IA') \
                         GROUP BY state, year) a \
                   LEFT OUTER JOIN \
                        (SELECT state, year, count(*) as cnt \
                         FROM unified \
                         WHERE state IN('NE','IA') \
                         GROUP BY state, year) b \
                    ON a.state = b.state \
                    AND a.year = b.year \
                    ORDER BY a.state, a.year"))

State,Year,Elementary,Unified
IA,2017,336,336
IA,2018,333,333
NE,2017,251,251
NE,2018,246,246
